In [1]:
#export

from exp.nb_01 import *
from torch import nn, optim, tensor, Tensor
from torch.utils.data import TensorDataset, Dataset, DataLoader

In [2]:
#export
data_path=Path(r'd:\git\dl\data')
mnist_path=data_path.joinpath('mnist.pkl.gz')
Path.ls=lambda x: L(x.iterdir())

In [3]:
data_path.ls()

(#3) [d:\git\dl\data\imagenette2-160,d:\git\dl\data\imagewoof2-160,d:\git\dl\data\mnist.pkl.gz]

In [4]:
with gzip.open(mnist_path, 'rb') as f:
    ((x_train, y_train), (x_valid, y_valid), _)=pickle.load(f, encoding='latin-1')
    
x_train, y_train, x_valid, y_valid=map(tensor, (x_train, y_train, x_valid, y_valid))

In [5]:
x_train[0].shape

torch.Size([784])

In [6]:
idx=x_train[0].nonzero()[0]
x_train[0][idx-3: idx+5]

tensor([0.0000, 0.0000, 0.0000, 0.0117, 0.0703, 0.0703, 0.0703, 0.4922])

In [7]:
bs=128
epochs=3
lr=0.1

In [8]:
train_ds=TensorDataset(x_train, y_train)
valid_ds=TensorDataset(x_valid, y_valid)

In [9]:
#export
Rank0Tensor = NewType('OneEltTensor', Tensor)
LossFunction = Callable[[Tensor, Tensor], Rank0Tensor]
Model = nn.Module

def is_listy(x: Any)->bool: return isinstance(x, (tuple, list))

def loss_batch(model: Model, xb:Tensor, yb:Tensor, 
               loss_fn:LossFunction,opt: optim.Optimizer=None):
    if not is_listy(xb): xb= [xb]
    if not is_listy(yb): yb= [yb]
    loss= loss_fn(model(*xb), (*yb))
    if opt is not None:
        loss.backward()
        opt.step()
        opt.zero_grad()
    return loss.item(), len(yb)

In [10]:
#export
def fit(epochs:int, model:Model, loss_fn:LossFunction, 
        opt:optim.Optimizer, train_dl:DataLoader, valid_dl:DataLoader):
    "Train `model` on `train_dl` with `optim` then validate against `valid_dl`"
    for epoch in range(epochs):
        model.train()
        for xb,yb in train_dl: loss,_ = loss_batch(model, xb, yb, loss_fn, opt)

        model.eval()
        with torch.no_grad():
            losses,nums = zip(*[loss_batch(model, xb, yb, loss_fn)
                                for xb,yb in valid_dl])
        val_loss = np.sum(np.multiply(losses,nums)) / np.sum(nums)

        print(epoch, val_loss)

In [11]:
#export
LambdaFunc = Callable[[Tensor],Tensor]
class Lambda(nn.Module):
    "An easy way to create a pytorch layer for a simple `func`"
    def __init__(self, func:LambdaFunc):
        "create a layer that simply calls `func` with `x`"
        super().__init__()
        self.func=func
        
    def forward(self, x): return self.func(x)

In [12]:
#export
def noop(x): return x

def ResizeBatch(*size:int) -> Tensor: 
    "Layer that resizes x to `size`, good for connecting mismatched layers"
    return Lambda(lambda x: x.view((-1,)+size))
def Flatten()->Tensor: 
    "Flattens `x` to a single dimension, often used at the end of a model"
    return Lambda(lambda x: x.view((x.size(0), -1)))
def PoolFlatten()->nn.Sequential:
    "Apply `nn.AdaptiveAvgPool2d` to `x` and then flatten the result"
    return nn.Sequential(nn.AdaptiveAvgPool2d(1), Flatten())

def conv2d(ni:int, nf:int, ks:int=3, stride:int=1, padding:int=None, bias=False) -> nn.Conv2d:
    "Create `nn.Conv2d` layer: `ni` inputs, `nf` outputs, `ks` kernel size. `padding` defaults to `k//2`"
    if padding is None: padding = ks//2
    return nn.Conv2d(ni, nf, kernel_size=ks, stride=stride, padding=padding, bias=bias)

def conv2d_relu(ni:int, nf:int, ks:int=3, stride:int=1, 
                padding:int=None, bn:bool=False) -> nn.Sequential:
    "Create a `conv2d` layer with `nn.ReLU` activation and optional(`bn`) `nn.BatchNorm2d`"
    layers = [conv2d(ni, nf, ks=ks, stride=stride, padding=padding), nn.ReLU()]
    if bn: layers.append(nn.BatchNorm2d(nf))
    return nn.Sequential(*layers)

def conv2d_trans(ni:int, nf:int, ks:int=2, stride:int=2, padding:int=0) -> nn.ConvTranspose2d:
    "Create `nn.nn.ConvTranspose2d` layer: `ni` inputs, `nf` outputs, `ks` kernel size. `padding` defaults to 0"
    return nn.ConvTranspose2d(ni, nf, kernel_size=ks, stride=stride, padding=padding)

In [13]:
model = nn.Sequential(
   nn.Linear(784, 512),
   nn.Linear(512, 256),
   nn.Linear(256, 56),
   nn.Linear(56, 10) 
)

In [14]:
def get_data(train_ds, valid_ds, bs):
    return (DataLoader(train_ds, batch_size=bs, shuffle=True),
            DataLoader(valid_ds, batch_size=bs*2))

train_dl,valid_dl = get_data(train_ds, valid_ds, bs)

In [15]:
loss_fn = F.cross_entropy

In [16]:
opt = optim.SGD(model.parameters(), lr=lr)

In [17]:
loss_fn(model(x_valid[0:bs]), y_valid[0:bs])

tensor(2.3231, grad_fn=<NllLossBackward>)

In [18]:
fit(epochs, model, loss_fn, opt, train_dl, valid_dl)

0 0.30990973114967346
1 0.2855317899957299
2 0.2888068364933133


In [19]:
def mnist2img(x): return x.view(1, 28, 28)

In [20]:
#export

@dataclass
class DatasetTfm(Dataset):
    'Applying transform to the pipelines'
    ds: Dataset
    tfm: Callable=None
        
    def __len__(self): return len(self.ds)
    
    def __getitem__(self, idx: int):
        x,y = self.ds[idx]
        if self.tfm is not None: x=self.tfm(x)
        return x, y

In [21]:
train_tds=DatasetTfm(train_ds, mnist2img)
valid_tds=DatasetTfm(valid_ds, mnist2img)

In [22]:
x,y = next(iter(valid_tds))

In [25]:
mnist_path

WindowsPath('d:/git/dl/data/mnist.pkl.gz')

In [26]:
valid_ds[0][0].shape, x[0].shape

(torch.Size([784]), torch.Size([28, 28]))

In [27]:
torch.allclose(valid_ds[0][0], x[0].view(-1))

True

In [28]:
def get_data(train_ds, valid_ds, bs):
    return (DataLoader(train_ds, batch_size=bs, shuffle=True),
           DataLoader(valid_ds, batch_size=bs*2, shuffle=False))

train_dl, valid_ds=get_data(train_tds, valid_tds, 128)

In [29]:
#export
def ifnone(a:bool, b:any):
    return b if a is None else a

In [30]:
#export

default_device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
Tensors = Union[Tensor, Collection['Tensors']]

def to_device(b:Tensors, device: torch.device):
    device= ifnone(device, default_device)
    if is_listy(b): return [to_device(o, device) for o in b]
    return b.to(device)

In [31]:
#export

@dataclass
class DeviceDataLoader():
    "`DataLoader` that ensures batches from `dl` are on `device`"
    dl: DataLoader
    device: torch.device

    def __len__(self) -> int: return len(self.dl)
    def proc_batch(self,b:Tensors): return to_device(b, self.device)

    def __iter__(self)->Tensors:
        "Ensure batches from `dl` are on `device` as we iterate"
        self.gen = map(self.proc_batch, self.dl)
        return iter(self.gen)

    @classmethod
    def create(cls, *args, device:torch.device=default_device, **kwargs): return cls(DataLoader(*args, **kwargs), device=device)

In [32]:
def get_data(train_ds, valid_ds, bs):
    return(DeviceDataLoader.create(train_ds, bs, shuffle=True),
          DeviceDataLoader.create(valid_ds, bs*2, shuffle=False))

train_dl, valid_dl= get_data(train_tds, valid_tds, bs)

In [33]:
#export
def simple_cnn(actns:Collection[int], kernel_szs:Collection[int], 
               strides:Collection[int]) -> nn.Sequential:
    "CNN with `conv2d_relu` layers defined by `actns`, `kernel_szs` and `strides`"
    layers = [conv2d_relu(actns[i], actns[i+1], kernel_szs[i], stride=strides[i])
        for i in range(len(strides))]
    layers.append(PoolFlatten())
    return nn.Sequential(*layers)

In [34]:
def get_model():
    model = simple_cnn([1,16,16,10], [3,3,3], [2,2,2]).to(default_device)
    return model, optim.SGD(model.parameters(), lr=lr)

In [35]:
model, opt=get_model()

In [36]:
#export
def fit(epochs:int, model:Model, loss_fn:LossFunction, 
        opt:optim.Optimizer, train_dl:DataLoader, valid_dl:DataLoader) -> None:
    "Train `model` for `epochs` with `loss_fun` and `optim`"
    for epoch in range(epochs):
        model.train()
        for xb,yb in train_dl: loss,_ = loss_batch(model, xb, yb, loss_fn, opt)

        model.eval()
        with torch.no_grad():
            losses,nums = zip(*[loss_batch(model, xb, yb, loss_fn)
                                for xb,yb in valid_dl])
        val_loss = np.sum(np.multiply(losses,nums)) / np.sum(nums)

        print(epoch, val_loss)

In [37]:
fit(epochs, model, loss_fn, opt, train_dl, valid_dl)

0 2.2466877341270446
1 1.362467873096466
2 1.1592233553528786


In [38]:
#export
TItem = TypeVar('TItem')
TfmCallable = Callable[[TItem],TItem]
TfmList = Union[TfmCallable, Collection[TfmCallable]]
Tfms = Optional[TfmList]

@dataclass
class DataBunch():
    "Bind `train_dl`, `valid_dl` to `device`"
    train_dl:DataLoader
    valid_dl:DataLoader
    device:torch.device=None

    @classmethod
    def create(cls, train_ds:Dataset, valid_ds:Dataset, bs:int=64, 
               train_tfm:Tfms=None, valid_tfm:Tfms=None, device:torch.device=None, **kwargs):
        return cls(DeviceDataLoader.create(DatasetTfm(train_ds, train_tfm), bs,   shuffle=True,  device=device, **kwargs),
                   DeviceDataLoader.create(DatasetTfm(valid_ds, valid_tfm), bs*2, shuffle=False, device=device, **kwargs),
                   device=device)

class Learner():
    "Train `model` on `data` for `epochs` using learning rate `lr` and `opt_fn` to optimize training"
    def __init__(self, data:DataBunch, model:Model):
        self.data,self.model = data,to_device(model, data.device)

    def fit(self, epochs, lr, opt_fn=optim.SGD):
        opt = opt_fn(self.model.parameters(), lr=lr)
        loss_fn = F.cross_entropy
        fit(epochs, self.model, loss_fn, opt, self.data.train_dl, self.data.valid_dl)

In [39]:
data = DataBunch.create(train_ds, valid_ds, bs=bs, train_tfm=mnist2img, valid_tfm=mnist2img)
model = simple_cnn([1,16,16,10], [3,3,3], [2,2,2])
learner = Learner(data, model)
opt_fn = partial(optim.SGD, momentum=0.9)

In [43]:
learner.fit(1, lr/5, opt_fn=opt_fn)

TypeError: 'DataLoader' object is not subscriptable

In [ ]:
learner.fit(2, lr, opt_fn=opt_fn)
learner.fit(1, lr/5, opt_fn=opt_fn)

In [44]:
#export
def ifnone(a:bool,b:Any):
    "`a` if its not None, otherwise `b`"
    return b if a is None else a

default_device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
Tensors = Union[Tensor, Collection['Tensors']]

def to_device(b:Tensors, device:torch.device):
    "Ensure `b` is on `device`"
    device = ifnone(device, default_device)
    if is_listy(b): return [to_device(o, device) for o in b]
    return b.to(device)

@dataclass
class DeviceDataLoader():
    "`DataLoader` that ensures batches from `dl` are on `device`"
    dl: DataLoader
    device: torch.device

    def __len__(self) -> int: return len(self.dl)
    def proc_batch(self,b:Tensors): return to_device(b, self.device)

    def __iter__(self)->Tensors:
        "Ensure batches from `dl` are on `device` as we iterate"
        self.gen = map(self.proc_batch, self.dl)
        return iter(self.gen)

    @classmethod
    def create(cls, *args, device:torch.device=default_device, **kwargs): return cls(DataLoader(*args, **kwargs), device=device)

In [45]:
def get_data(train_ds, valid_ds, bs):
    return (DeviceDataLoader.create(train_ds, bs,   shuffle=True),
            DeviceDataLoader.create(valid_ds, bs*2, shuffle=False))

In [46]:
train_dl,valid_dl = get_data(train_tds, valid_tds, bs)

In [47]:
def get_model():
    model = simple_cnn([1,16,16,10], [3,3,3], [2,2,2]).to(default_device)
    return model, optim.SGD(model.parameters(), lr=lr)

In [48]:
model,opt = get_model()

In [49]:
train_dl,valid_dl = get_data(train_tds, valid_tds, bs)

In [50]:
x.type(),y.type()

('torch.FloatTensor', 'torch.LongTensor')

In [51]:
#export
def fit(epochs:int, model:Model, loss_fn:LossFunction, 
        opt:optim.Optimizer, train_dl:DataLoader, valid_dl:DataLoader) -> None:
    "Train `model` for `epochs` with `loss_fun` and `optim`"
    for epoch in range(epochs):
        model.train()
        for xb,yb in train_dl: loss,_ = loss_batch(model, xb, yb, loss_fn, opt)

        model.eval()
        with torch.no_grad():
            losses,nums = zip(*[loss_batch(model, xb, yb, loss_fn)
                                for xb,yb in valid_dl])
        val_loss = np.sum(np.multiply(losses,nums)) / np.sum(nums)

        print(epoch, val_loss)

In [52]:
fit(epochs, model, loss_fn, opt, train_dl, valid_dl)

0 2.2894701957702637
1 1.4860345005989075
2 0.9555125489830971


In [53]:
#export
TItem = TypeVar('TItem')
TfmCallable = Callable[[TItem],TItem]
TfmList = Union[TfmCallable, Collection[TfmCallable]]
Tfms = Optional[TfmList]

@dataclass
class DataBunch():
    "Bind `train_dl`, `valid_dl` to `device`"
    train_dl:DataLoader
    valid_dl:DataLoader
    device:torch.device=None

    @classmethod
    def create(cls, train_ds:Dataset, valid_ds:Dataset, bs:int=64, 
               train_tfm:Tfms=None, valid_tfm:Tfms=None, device:torch.device=None, **kwargs):
        return cls(DeviceDataLoader.create(DatasetTfm(train_ds, train_tfm), bs,   shuffle=True,  device=device, **kwargs),
                   DeviceDataLoader.create(DatasetTfm(valid_ds, valid_tfm), bs*2, shuffle=False, device=device, **kwargs),
                   device=device)

class Learner():
    "Train `model` on `data` for `epochs` using learning rate `lr` and `opt_fn` to optimize training"
    def __init__(self, data:DataBunch, model:Model):
        self.data,self.model = data,to_device(model, data.device)

    def fit(self, epochs, lr, opt_fn=optim.SGD):
        opt = opt_fn(self.model.parameters(), lr=lr)
        loss_fn = F.cross_entropy
        fit(epochs, self.model, loss_fn, opt, self.data.train_dl, self.data.valid_dl)

In [55]:
data = DataBunch.create(train_ds, valid_ds, bs=bs, train_tfm=mnist2img, valid_tfm=mnist2img)
model = simple_cnn([1,16,16,10], [3,3,3], [2,2,2])
learner = Learner(data, model)
opt_fn = partial(optim.SGD, momentum=0.9)

In [56]:
learner.fit(1, lr/5, opt_fn=opt_fn)

TypeError: 'DataLoader' object is not subscriptable